In [1]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

Matplotlib is building the font cache; this may take a moment.


In [2]:
#Loading the dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

# Pad sequesnces to same length
x_train = pad_sequences(x_train, maxlen=256)
x_test = pad_sequences(x_test, maxlen=256)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [6]:
#Building the model

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense

model = Sequential([
        Embedding(input_dim=10000,output_dim=16),
        GlobalAveragePooling1D(),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
])


In [7]:
#Compile and train

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=10, batch_size=512, validation_split=0.2)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.5630 - loss: 0.6915 - val_accuracy: 0.6326 - val_loss: 0.6818
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6547 - loss: 0.6765 - val_accuracy: 0.7552 - val_loss: 0.6547
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7317 - loss: 0.6460 - val_accuracy: 0.7568 - val_loss: 0.6094
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7648 - loss: 0.5942 - val_accuracy: 0.7954 - val_loss: 0.5495
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8117 - loss: 0.5312 - val_accuracy: 0.8090 - val_loss: 0.4913
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8361 - loss: 0.4691 - val_accuracy: 0.8312 - val_loss: 0.4431
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8383 - loss: 0.4225 - val_accuracy: 0.8362 - val_loss: 0.4105
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8598 - loss: 0.3844 - val_accuracy: 0.8526 - v

In [8]:
#Evaluate

results = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {results}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8587 - loss: 0.3495
Test Accuracy: [0.3531104624271393, 0.8573200106620789]


In [10]:
#Validation Metrics

from sklearn.metrics import precision_score, recall_score, f1_score

y_pred = (model.predict(x_test)>0.5).astype("int32")

percision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Precision: {percision}, Recall: {recall}, F1 Score: {f1}')

782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Precision: 0.8873471511577486, Recall: 0.81856, F1 Score: 0.8515667263118472


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label = 'val acc')
plt.legend()
plt.show()